# WORKFLOW SUMMARY

1. DATA CLEAN: 
* load data 
* drop empty columns 
* drop duplicate rows
* fill in missing values
* add two boolean features: Subject_is_empty, Body_is_empty

2. Text Language Detection & Change
* subject_lang
* body_lang
* subject_en
* body_en

3. INFORMATION EXTRACTION:
* subject_links (then dropped, since it's empty)
* subject_emails
* subject_attachments
* subject_special_chars
* remained_subject
* body_links
* body_emails
* body_attachments
* body_special_chars
* body_subject

4. Tools:
* Panda -> load data, manipulate data, create new features
* re -> pattern matching, extract structured data(emails, URLs, links, special characters)
* langdetect -> detect the language of the Subject and Body
* googletrans -> transform non-English text into English

5. Outputs:
extract_df_2 -> cleaned_email_data.csv

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('CaptstoneProjectData_2025.csv')

# Observe data

In [3]:
df.head()

,Subject,Body,Unnamed: 2,Unnamed: 3
0,®Review your shipment details / Shipment Notif...,Notice: This message was sent from outside the...,NaN,NaN
1,Υоur ассоunt іѕ оn hоld,\r\nVotre réponse a bien été prise en compte.\...,NaN,NaN
2,Completed: Invoice # KZ89TYS2564 from-Bestbuy....,Notice: This message was sent from outside the...,NaN,NaN
3,UVic IMPORTANT NOTICE!,Your UVIC account has been filed under the lis...,NaN,NaN
4,You have (6) Suspended incoming messages,\r\n\r\nMessage generated from uvic.ca source...,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2576 entries, 0 to 2575
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Subject     2467 non-null   object 
 1   Body        2571 non-null   object 
 2   Unnamed: 2  0 non-null      float64
 3   Unnamed: 3  0 non-null      float64
dtypes: float64(2), object(2)
memory usage: 80.6+ KB


# Deal with missing/duplicate data

In [5]:
# Remove columns with all null values
valid_df = df.dropna(axis=1, how='all')
valid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2576 entries, 0 to 2575
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Subject  2467 non-null   object
 1   Body     2571 non-null   object
dtypes: object(2)
memory usage: 40.4+ KB


In [6]:
# Check for duplicate rows
valid_df.duplicated().any()

np.True_

In [7]:
# Drop duplicate records
valid_df = valid_df.drop_duplicates()
valid_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2351 entries, 0 to 2575
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Subject  2250 non-null   object
 1   Body     2347 non-null   object
dtypes: object(2)
memory usage: 55.1+ KB


In [8]:
# Check for missing values (NaNs)
valid_df.isna().sum()

Subject    101
Body         4
dtype: int64

In [9]:
# Impute missing values
valid_df['Subject'] = valid_df['Subject'].fillna('')
valid_df['Body'] = valid_df['Body'].fillna('')
valid_df['Subject_is_empty'] = valid_df['Subject'] == ''
valid_df['Body_is_empty'] = valid_df['Body'] == ''

In this step, missing values in the Subject and Body columns are imputed using empty strings (''). This ensures that all entries are string-type and avoids errors during text processing or vectorization. Instead of dropping the rows, we preserve them, as empty subjects or bodies may indicate phishing patterns.

Additionally, two new boolean columns, Subject_is_empty and Body_is_empty, are created to explicitly mark which entries were originally missing or empty. These flags can serve as useful features during pattern recognition or machine learning, as phishing emails often have missing or blank content fields.

In [10]:
valid_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2351 entries, 0 to 2575
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Subject           2351 non-null   object
 1   Body              2351 non-null   object
 2   Subject_is_empty  2351 non-null   bool  
 3   Body_is_empty     2351 non-null   bool  
dtypes: bool(2), object(2)
memory usage: 59.7+ KB


In [34]:
valid_df.head(10)

,Subject,Body,Subject_is_empty,Body_is_empty
0,®Review your shipment details / Shipment Notif...,Notice: This message was sent from outside the...,False,False
1,Υоur ассоunt іѕ оn hоld,\r\nVotre réponse a bien été prise en compte.\...,False,False
2,Completed: Invoice # KZ89TYS2564 from-Bestbuy....,Notice: This message was sent from outside the...,False,False
3,UVic IMPORTANT NOTICE!,Your UVIC account has been filed under the lis...,False,False
4,You have (6) Suspended incoming messages,\r\n\r\nMessage generated from uvic.ca source...,False,False
5,Account validation for uvic.ca user(s),"Hi helpdesk,\r\n\r\nTo continue using your ema...",False,False
6,Y‌our p‌ayme‌nt m‌et‌hod h‌as b‌een Dec‌li‌ned...,Notice: This message was sent from outside the...,False,False
7,Microsoft account security notification,Notice: This message was sent from outside the...,False,False
8,Urgent :AutoPay Payment was Unsuccessful !,[https://www.telstra.com.au/content/dam/tcom/a...,False,False
9,Verify your identity,Notice: This message was sent from outside the...,False,False


# Language normalization

Since we saw some non-english text in Subject and Body, we use langdetect to identify the language of the subject and body seperately

In [19]:
! pip install langdetect

In [ ]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

In [40]:
extract_df = valid_df.copy()
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return 'unknown'

extract_df['subject_lang'] = extract_df['Subject'].apply(detect_language)
extract_df['body_lang'] = extract_df['Body'].apply(detect_language)

In [32]:
extract_df.head()

,Subject,Body,Subject_is_empty,Body_is_empty,subject_lang,body_lang
0,®Review your shipment details / Shipment Notif...,Notice: This message was sent from outside the...,False,False,fr,en
1,Υоur ассоunt іѕ оn hоld,\r\nVotre réponse a bien été prise en compte.\...,False,False,et,et
2,Completed: Invoice # KZ89TYS2564 from-Bestbuy....,Notice: This message was sent from outside the...,False,False,en,en
3,UVic IMPORTANT NOTICE!,Your UVIC account has been filed under the lis...,False,False,en,en
4,You have (6) Suspended incoming messages,\r\n\r\nMessage generated from uvic.ca source...,False,False,en,en


use googletrans to translate non-English to English

In [22]:
! pip install googletrans==4.0.0-rc1

In [23]:
from googletrans import Translator

In [41]:
translator = Translator()

def translate_if_not_english(text, lang_code):
    try:
        if lang_code == 'en':
            return text
        result = translator.translate(text, dest='en')
        return result.text
    except:
        return text  # fallback to original

extract_df['subject_en'] = extract_df.apply(
    lambda row: translate_if_not_english(row['Subject'], row['subject_lang']), axis=1
)

extract_df['body_en'] = extract_df.apply(
    lambda row: translate_if_not_english(row['Body'], row['body_lang']), axis=1
)


In [50]:
extract_df.head()

,Subject,Body,Subject_is_empty,Body_is_empty,subject_lang,body_lang,subject_en,body_en
0,®Review your shipment details / Shipment Notif...,Notice: This message was sent from outside the...,False,False,fr,en,®REVIEW your Shipment Details / Shipment Notif...,Notice: This message was sent from outside the...
1,Υоur ассоunt іѕ оn hоld,\r\nVotre réponse a bien été prise en compte.\...,False,False,uk,et,Our Assound I-ON HOLD,Votre réponse a bien été prise en compte.\r\n[...
2,Completed: Invoice # KZ89TYS2564 from-Bestbuy....,Notice: This message was sent from outside the...,False,False,en,en,Completed: Invoice # KZ89TYS2564 from-Bestbuy....,Notice: This message was sent from outside the...
3,UVic IMPORTANT NOTICE!,Your UVIC account has been filed under the lis...,False,False,en,en,UVic IMPORTANT NOTICE!,Your UVIC account has been filed under the lis...
4,You have (6) Suspended incoming messages,\r\n\r\nMessage generated from uvic.ca source...,False,False,en,en,You have (6) Suspended incoming messages,\r\n\r\nMessage generated from uvic.ca source...


In [51]:
extract_df_1 = extract_df.copy()

# Feature Extraction

In [53]:
extract_df_2 = extract_df.copy()

In [55]:
# Extract email addresses
def extract_emails(text):
    return re.findall(r'\b[\w\.-]+?@\w+?\.\w+?\b', text)

# Extract URLs
def extract_urls(text):
    return re.findall(r'http[s]?://[^\s]+', text)

# Detect attachment phrases or common extensions
def extract_attachment_keywords(text):
    attachment_pattern = r'[\w\-.]+\.(?:zip|exe|jpg|jpeg|png|pdf|docx|doc|xls|xlsx|ppt|pptx|rar|7z)'
    attachments = re.findall(attachment_pattern, text, flags=re.IGNORECASE)
    return list(set(attachments))

# Extract special characters
def extract_special_characters(text):
    return re.findall(r'[^\w\s]', text)

# Remove extracted patterns from text
def remove_extracted(text, patterns):
    for pattern in patterns:
        text = text.replace(pattern, '')
    return text.strip()

# Subject extractions
# extract_df_2['subject_links'] = extract_df_2['subject_en'].apply(extract_urls)
extract_df_2['subject_emails'] = extract_df_2['subject_en'].apply(extract_emails)
extract_df_2['subject_attachments'] = extract_df_2['subject_en'].apply(extract_attachment_keywords)
extract_df_2['subject_special_chars'] = extract_df_2['subject_en'].apply(extract_special_characters)

extract_df_2['remained_subject_en'] = extract_df_2.apply(
    lambda row: remove_extracted(
        row['subject_en'],
        row['subject_emails'] + row['subject_attachments'] + row['subject_special_chars']
    ),
    axis=1
)

# Body extractions
extract_df_2['body_links'] = extract_df_2['body_en'].apply(extract_urls)
extract_df_2['body_emails'] = extract_df_2['body_en'].apply(extract_emails)
extract_df_2['body_attachments'] = extract_df_2['body_en'].apply(extract_attachment_keywords)
extract_df_2['body_special_chars'] = extract_df_2['body_en'].apply(extract_special_characters)

extract_df_2['remained_body_en'] = extract_df_2.apply(
    lambda row: remove_extracted(
        row['body_en'],
        row['body_links'] + row['body_emails'] + row['body_attachments'] + row['body_special_chars']
    ),
    axis=1
)
extract_df_2['remained_body_en'] = extract_df_2['remained_body_en'].str.replace(r'[\r\n]+', ' ', regex=True)



In [57]:
extract_df_2.head(10)

,Subject,Body,Subject_is_empty,Body_is_empty,subject_lang,body_lang,subject_en,body_en,subject_emails,subject_attachments,subject_special_chars,remained_subject_en,body_links,body_emails,body_attachments,body_special_chars,remained_body_en
0,®Review your shipment details / Shipment Notif...,Notice: This message was sent from outside the...,False,False,fr,en,®REVIEW your Shipment Details / Shipment Notif...,Notice: This message was sent from outside the...,[],[],"[®, /, /, :]",REVIEW your Shipment Details Shipment Notific...,[https://www.canadapost-postescanada.ca/cpc/as...,"[amuench@uvic.ca, hudsonesajoyce@gmail.com]",[],"[:, ., ., [, :, /, /, ., -, ., /, /, /, /, /, ...",Notice This message was sent from outside the ...
1,Υоur ассоunt іѕ оn hоld,\r\nVotre réponse a bien été prise en compte.\...,False,False,uk,et,Our Assound I-ON HOLD,Votre réponse a bien été prise en compte.\r\n[...,[],[],[-],Our Assound ION HOLD,[https://www.edigitalagence.com.au/wp-connet/u...,[foipp@uvic.ca],[netflix-logo-red-black-png.png],"[., [, :, /, /, ., ., ., /, -, /, /, -, -, -, ...",Votre réponse a bien été prise en compte Υоur ...
2,Completed: Invoice # KZ89TYS2564 from-Bestbuy....,Notice: This message was sent from outside the...,False,False,en,en,Completed: Invoice # KZ89TYS2564 from-Bestbuy....,Notice: This message was sent from outside the...,[],[],"[:, #, -, ., +, ., ., .]",Completed Invoice KZ89TYS2564 fromBestbuycom ...,[https://NA4.docusign.net/member/Images/email/...,"[auwaluu.ma.r.bu.ba@googlemail.com, icon-Downl...","[protect.doc, www.doc, support.doc, docComplet...","[:, ., ., [, ], [, :, /, /, ., ., /, /, /, /, ...",Notice This message was sent from outside the ...
3,UVic IMPORTANT NOTICE!,Your UVIC account has been filed under the lis...,False,False,en,en,UVic IMPORTANT NOTICE!,Your UVIC account has been filed under the lis...,[],[],[!],UVic IMPORTANT NOTICE,[https://forms.gle/TpBxJ1SRFwgYMd8c7>],[],[],"[/, ., ., ., <, :, /, /, ., /, >, -, ., ., ,]",Your UVIC account has been filed under the lis...
4,You have (6) Suspended incoming messages,\r\n\r\nMessage generated from uvic.ca source...,False,False,en,en,You have (6) Suspended incoming messages,\r\n\r\nMessage generated from uvic.ca source...,[],[],"[(, )]",You have 6 Suspended incoming messages,[https://googleweblight.com/i?u=https://cloudf...,"[helpdesk@uvic.ca, helpdesk@uvic.ca]",[],"[., ., ., :, @, ., (, ), <, :, /, /, ., /, ?, ...",Message generated from uvicca source Sender ...
5,Account validation for uvic.ca user(s),"Hi helpdesk,\r\n\r\nTo continue using your ema...",False,False,en,en,Account validation for uvic.ca user(s),"Hi helpdesk,\r\n\r\nTo continue using your ema...",[],[],"[., (, )]",Account validation for uvicca users,[https://apiservices.krxd.net/click_tracker/tr...,[helpdesk@uvic.ca],[],"[,, (, @, ., ), ,, ., <, :, /, /, ., ., /, /, ...",Hi helpdesk To continue using your email accou...
6,Y‌our p‌ayme‌nt m‌et‌hod h‌as b‌een Dec‌li‌ned...,Notice: This message was sent from outside the...,False,False,en,en,Y‌our p‌ayme‌nt m‌et‌hod h‌as b‌een Dec‌li‌ned...,Notice: This message was sent from outside the...,[],[],"[‌, ‌, ‌, ‌, ‌, ‌, ‌, ‌, ‌, ., ‌, #, ‌, -]",Your payment method has been Declined CaseID 9...,[https://script.google.com/macros/s/AKfycbwUfF...,[],[],"[:, ., ., ., ., :, *, *, ., ., ., <, :, /, /, ...",Notice This message was sent from outside the ...
7,Microsoft account security notification,Notice: This message was sent from outside the...,False,False,en,en,Microsoft account security notification,Notice: This message was sent from outside the...,[],[],[],Microsoft account security notification,[https://go.microsoft.com/fwlink/?LinkId=20867...,"[a@hotmail.com, a@hotmail.com]",[],"[:, ., ., /, /, ,, *, *, @, ., <, :, *, *, @, ...",Notice This message was sent from outside the ...
8,Urgent :AutoPay Payment was Unsuccessful !,[https://www.telstra.com.au/content/dam/tcom/a...,False,False,en,en,Urgent :AutoPay Payment was Unsuccessful !,[https://www.telstra.com

Note: there are emails and attachments in Subject, while there is no links in Subject.

In [62]:
extract_df_2.loc[extract_df_2['subject_emails'].apply(lambda x: len(x) > 0), 'subject_emails']

77                 [helpdesk@uvic.ca]
324     [drsudhirruparelia@gmail.com]
356                [helpdesk@uvic.ca]
364                    [vpac@uvic.ca]
365                    [vpac@uvic.ca]
                    ...              
2434    [procurementofficer4@uvic.ca]
2475                    [ljh@uvic.ca]
2491              [rpkmadmin@uvic.ca]
2516                [biocoop@uvic.ca]
2522                [hisgrad@uvic.ca]
Name: subject_emails, Length: 198, dtype: object

In [63]:
extract_df_2.loc[extract_df_2['subject_attachments'].apply(lambda x: len(x) > 0), 'subject_attachments']

89                [accessment.docx]
93                   [Updates.docx]
119                    [Bonus.docx]
297                 [Agreement.Pdf]
671                 [Agreement.pdf]
770     [ihg_logo_folio7253703.pdf]
2296              [EVALUATION.docx]
2438              [EVALUATION.docx]
2537              [.1055043768.xls]
Name: subject_attachments, dtype: object

# Output result

In [65]:
extract_df_2.to_csv("cleaned_email_data.csv", index=False, encoding='utf-8-sig')